# Run the pipeline from the command line

The JWST pipeline can either be executed on the command line with `strun` or in Python, first we will execute it from the command line.

`strun` takes as the first argument a configuration file and as second argument the data.

## Prepare the configuration file

`stpipe` has many analysis modules already implemented, see the Github repository:

* <https://github.com/spacetelescope/jwst/tree/master/jwst>

moreover it has some complete pipelines which execute a set of modules and pipe the output of a module to the next without ever writing it to disk.

In this example we use the `jwst.pipeline.Image2Pipeline` pipeline which is the Level 2 processing pipeline for imagers. It is composed of 5 steps:

* `bkg_subtract`
* `assign_wcs`
* `flat_field`
* `photom`
* `resample`

We can configure it by writing a configuration file, in particular we can set `skip=True` to disable the steps we are not interested in.

We can also have **nested** configuration files, for example for flat-fielding, we can then write the `image2.cfg` configuration file:

In [3]:
%%writefile image2.cfg

name = "Image2Pipeline"
class = "jwst.pipeline.Image2Pipeline"
save_results = True

    [steps]
      [[bkg_subtract]]
      [[assign_wcs]]
        skip = True
      [[flat_field]]
        config_file = flat_field.cfg
      [[photom]]
        skip = True
      [[resample]]
        skip = True

Overwriting image2.cfg


We then have a dedicated configuration file for flat fielding.
By default the pipeline would connect to the CDRS (the JWST database) to get a suitable flat for the current exposure, but we can override that with a file that we provide:

In [4]:
%%writefile flat_field.cfg

name = "flat_field" 
class = "jwst.flatfield.FlatFieldStep"

# Optional filename suffix for output flats (only for MOS data).
flat_suffix = None
override_flat = 'flat.fits'

Writing flat_field.cfg


## Prepare the input data association

The second argument to the pipeline is generally an input file. In our case however, we need 2 (or more) input files, therefore we can use an association, see the docs: <https://jwst-pipeline.readthedocs.io/en/latest/jwst/associations/overview.html>.

It basically is a Python dictionary that defines a set of inputs and their role in the current pipeline, it is written as a JSON file.

In [6]:
%%writefile asn_subtract_bg_flat.json
{
    "asn_rule": "Asn_Lv2Image",
    "asn_pool": "pool",
    "program": "82600",
    "asn_type": "image2",
    "products": [
        {
            "name": "test_iris_subtract_bg_flat",
            "members": [
                {
                    "expname": "sci.fits",
                    "exptype": "science"
                },
                {
                    "expname": "bg.fits",
                    "exptype": "background"
                }
            ]
        }
    ]
}

Overwriting asn_subtract_bg_flat.json


The pipeline uses these inputs for processing, for example if we provide more exposures with type "background", it is going to average them and apply sigma clipping.